In [1]:
import pandas as pd
import os

In [3]:
#import excel spreadsheet

filename_File1 = "data/udot/AADTHistory.xlsx"

xl_file1 = pd.ExcelFile(filename_File1)

dfs1 = {sheet_name: xl_file1.parse(sheet_name) for sheet_name in xl_file1.sheet_names}


c:\Users\bhereth\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [25]:
display(xl_file1.sheet_names)
dfs1['AADT_2021']

['AADT_2021', 'AADT_Rounded', 'AADT_Unrounded']

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT_2021
0,0006PM,0.000,46.038,474
1,0006PM,46.038,77.556,424
2,0006PM,77.556,82.897,609
3,0006PM,82.897,83.911,2272
4,0006PM,83.911,87.694,3852
...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886
4569,0015PC29501,0.000,0.166,1000
4570,0015PC30554,0.000,1.135,35025
4571,0092NC00101,0.000,3.294,12201


In [38]:
# prepare 2021 data

df2021 = dfs1['AADT_2021']

df2021['YEAR'] = 2021
df2021.rename(columns={'AADT_2021':'AADT'}, inplace=True)
df2021

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT,YEAR
0,0006PM,0.000,46.038,474,2021
1,0006PM,46.038,77.556,424,2021
2,0006PM,77.556,82.897,609,2021
3,0006PM,82.897,83.911,2272,2021
4,0006PM,83.911,87.694,3852,2021
...,...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886,2021
4569,0015PC29501,0.000,0.166,1000,2021
4570,0015PC30554,0.000,1.135,35025,2021
4571,0092NC00101,0.000,3.294,12201,2021


In [39]:
# melt AADT columns to rows

df = dfs1['AADT_Unrounded']

# assuming df is your DataFrame

# get all columns that start with 'AADT'
aadt_cols = [col for col in df.columns if col.startswith('AADT')]

# define the id columns that you want to keep
id_vars = ['Route', 'Beg MP', 'End MP']

# melt the DataFrame
dfUnder2021 = df.melt(id_vars=id_vars, value_vars=aadt_cols, var_name='YEAR', value_name='AADT')

# replace 'AADT' in the 'YEAR' column and convert to integer
dfUnder2021['YEAR'] = dfUnder2021['YEAR'].str.replace('AADT', '').astype(int)

dfUnder2021.rename(columns={'Route':'ROUTE_ID','Beg MP':'FROM_MEASURE','End MP':'TO_MEASURE'}, inplace=True)

display(dfUnder2021)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.000,46.038,2020,430
1,0006PM,46.038,77.556,2020,385
2,0006PM,77.556,82.897,2020,552
3,0006PM,82.897,83.911,2020,2061
4,0006PM,83.911,87.694,2020,3409
...,...,...,...,...,...
181475,3468PM,0.000,5.125,1981,0
181476,3469PM,0.000,6.930,1981,0
181477,3470PM,0.000,1.039,1981,0
181478,3478PM,0.000,2.040,1981,0


In [40]:
# combine 2021 and under 2021
dfAADT = pd.concat([dfUnder2021,df2021])
dfAADT

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.000,46.038,2020,430
1,0006PM,46.038,77.556,2020,385
2,0006PM,77.556,82.897,2020,552
3,0006PM,82.897,83.911,2020,2061
4,0006PM,83.911,87.694,2020,3409
...,...,...,...,...,...
4568,0015PC29402,0.000,0.199,2021,8886
4569,0015PC29501,0.000,0.166,2021,1000
4570,0015PC30554,0.000,1.135,2021,35025
4571,0092NC00101,0.000,3.294,2021,12201


In [ ]:
# get segments

sdf = pd.dataframe.spatial.from_featureclass('data/segments/Master_Segs_withFactors_20220915.shp')